# Outline
- review backend terms
- talk about docker vs VM
- create new branch of solvestack for updates to db (review git commands, branching)
- migrate db to postgresql
    - show how current sqlite file is set up
    - show how postgresql would be set up
    - talk about some additional features postgresql has
- review commands used to get web backend running
    - installing packages
    - creating folders
    - running the webserver for postgresql
- show how same commands translate to docker
    - same as above

# What's Docker?
Docker will save us a ton of time when it comes to getting our projects running, and making sure other people can run them as well!
- It does this by creating `containers`, which house running programs in an isolated manner
- The containers can house web services (like our django app) and databases (like we use for our web backend)
- Today, we'll go over the steps setting up these applications and then show how Docker can make that easier.  We'll also touch on postgresql and sqlite, since we'll be migrating the database during this.
- Below are the steps we have to go through when setting up projects locally
- Later, we'll set up the same steps but have Docker do them!

## Additional info
- https://docs.docker.com/get-started/
- https://docs.docker.com/engine/reference/builder/
- https://docs.docker.com/compose/django/
- https://www.postgresql.org/about/

## Quick Docker Demo
- run a demo container
- `docker run hello-world`
- `docker run -it ubuntu bash`

## DB Terms Review!
Let's go over some terms before we launch into today's project
- sql
- backend
- database
- ORM
- migrations
- model
- serializer
- URL resources endpoint
- Web server
- protocol
- request

# Steps set up locally
Create a new folder, and navigate to it in terminal with `cd`

Run `git clone https://github.com/SolveStack/solvestack_backend.git`

then `cd solvestack_backend`

We'll create a separate branch to do our work so we don't cause any problems with stable code:
`git checkout -b postgres_docker`

Now let's get our requirements set up for the virtual environment!

Check out steps in readme file:

*On fresh updates, do the following:*

```bash
source env/bin/activate
pip3 install -r dev-requirements.txt
pip3 install -r requirements.txt
```
We'll stop before running the next step.  We need to grab another file to replace the one we usually load - it'll have our terms from the last lecture.
- uncompress `solvestack.zip`
- copy the `solvestack` folder from the zip over the `solvestack` folder we have downloaded from git, merging files with existing ones
> this updates the model/admin files we played with the other weekend
- run `python manage.py makemigrations`
- run `python3 manage.py migrate`
- run `python3 manage.py loaddata solvestack/fixtures/db.json`
- run `python3 manage.py createsuperuser`


*To start the backend:*
```bash
python3 manage.py runserver
```

If we navigate to this [link](http://127.0.0.1:8000/admin/) now, we should see the admin page containing our glossary terms

# Wouldn't it be nice to not do that?
- we can use docker to do these steps for us!
- here's an example of a previous exercise...

# How could we use this in solvestack?
The solvestack web app needs different components to run - for our purposes today, we're focused on:

1. the web application server
1. the database server

If we're going to use Docker to launch and develop the solvestack webapp, it'd be nice to have it running a more robust database than sqlite.

# DBs commonly used in web apps
- MySQL
- PostgreSQL
- and many more...

## What makes these different?
- unlike SQLite, these databases run full servers, which can monitor database activity and scale up to larger sizes, as well as handle more traffic, than a single sqlite file could
- since we're building a web app, it'd be nice to have a development environment that more closely resembled the environment the app will run in for users
- postgresql is a very powerful and popular database, and we can migrate our sqlite file to it
- this will aid us in setting up a docker image later, and also allow us to demo what these components are like running together

# Transitioning from sqlite to postgresql
## Creating the postgresql db
You don't have to follow along with these steps if you don't have postgresql installed, but if you want to try postgresql can be downloaded and installed [here](https://www.postgresqltutorial.com/postgresql-getting-started/)
- create database in postgresql
    - launch psql
    - hit enter through prompts until password, then enter
    - `CREATE DATABASE solvestack_demo;`
    - `CREATE USER solvestack_demo_user WITH PASSWORD 'solvestack';`
    - `ALTER ROLE solvestack_demo_user SET client_encoding TO 'utf8';`
    - `ALTER ROLE solvestack_demo_user SET default_transaction_isolation TO 'read committed';`
    - `ALTER ROLE solvestack_demo_user SET timezone TO 'UTC';`
    - `GRANT ALL PRIVILEGES ON DATABASE solvestack_demo TO solvestack_demo_user;`
    - finally, type `\q` to exit

Show pgAdmin view

## Modify django settings.py and run!
Change the databases section to the code below, and our Django settings will match the database we just created:
```python
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.postgresql_psycopg2',
        'NAME': 'solvestack_docker_1',
        'USER': 'solvestack_docker_1_user',
        'PASSWORD': 'solvestack',
        'HOST': 'localhost',
        'PORT': '5432',
    }
}

- we changed the engine to use postgresql
- the remaining values match what we entered into psql when we created the db

```
- `pip3 install psycopg2` - we will add this to the requirements.txt later!
- run `python manage.py makemigrations`
- run `python manage.py migrate`
- run `python manage.py shell`

in shell:
```python
from django.contrib.contenttypes.models import ContentType
ContentType.objects.all().delete()
exit
```

How do we know we're using the new database?  Let's modify it!
- modify a record using django's admin page
- query in dbeaver for sqlite for comparison

## To Docker!
Now we need to take the steps we just did manually, and translate them to docker!

- what is docker?
- what are some docker terms we need to know?
    - container: a process that runs isolated from other processes
    - image: filesystem used for containers
    - https://docs.docker.com/engine/reference/builder/
- set up requirements files for install (folder, -r to point to base file)

requirements.txt
```
Django>=3.0
django-cors-headers>=3.5.0
django-extensions>=3.0.5
django-filter>=2.3.0
djangorestframework>=3.11.1
shortuuid>=1.0.1
psycopg2-binary>=2.8
```

dev_requirements.txt
```
black==19.3b0
-r requirements.txt
```

- create dockerfile
```python
    # The base image we want to inherit from
    FROM python:3
    ENV PYTHONUNBUFFERED 1
    ARG ENV=dev-requirements

    RUN mkdir /solvestack
    WORKDIR /solvestack

    # using ADD with the requirements (vs copy) makes use of caching to speed up consecutive builds
    ADD ./requirements requirements

    # Install the pip requirements file depending on 
    # the uncommitted .env file passed in when starting build.
    RUN pip install -Ur requirements/$ENV.txt

    # Copy the rest of our application.
    COPY . .

    # Expose the application on port 8000 (check if container run or compose needs to map to 8000 locally)
    EXPOSE 8000
    # Run test server
    CMD ["python", "manage.py", "runserver", "0.0.0.0:8000"]
```
    - go over elements/functions of the dockerfile
        - note how requirements are handled - we will be setting this up soon
        
    - create compose file
```python
    version: '3.8'

    services:

        db:
            image: postgres
            container_name: postgres
            env_file:
                - .env

        web:
            build: .
            command: python manage.py runserver 0.0.0.0:8000
            volumes:
                - .:/solvestack
            ports:
                - "8000:8000"
            depends_on:
                - db
```

- edit `setup.py`

```
    DATABASES = {
        'default': {
            'ENGINE': 'django.db.backends.postgresql_psycopg2',
            'NAME': 'solvestack_docker_1',
            'USER': 'solvestack_docker_1_user',
            'PASSWORD': 'solvestack',
            'HOST': 'db',
            'PORT': '5432',
        }
    }
```
- go over elements/functions of compose file
- modify requirements files
    - make folder called requirements
    - move both requirements files to requirements folder
    - add `psycopg2-binary>=2.8` to the bottom of `requirements.txt`
    - add `-r requirements.txt` to bottom of `dev_requirements.txt`
- .env files
    - modify with variables from https://hub.docker.com/_/postgres/
    - point to Dockerfile where these are referenced
- `docker-compose up --build`
    - note that the db succeeds but the website fails.  why is this?
    - let's take a look! open psql
    - log in to db, then pull `\l`
    - pull `\dt` - no tables yet
- `docker-compose down  --remove-orphans`

## Running one-time data through compose
These commands will set up the data in our container
- run `docker-compose run --service-ports web python manage.py makemigrations`
- run `docker-compose run --service-ports web python manage.py migrate`
- run `docker-compose run --service-ports web python manage.py loaddata solvestack/fixtures/db.json`
- run `docker-compose run --service-ports web python manage.py createsuperuser`